In [1]:
import minsearch

In [2]:
import json

In [4]:
with open('document.json', 'rt') as file_in:
    raw_docs = json.load(file_in)

In [5]:
documents = []

for course_dict in raw_docs:
    for doc in course_dict['documents']:
        doc['course'] = course_dict['course']
        documents.append(doc)

In [6]:
index = minsearch.Index(
    text_fields = ['question', 'text', 'section'],
    keyword_fields = ['course']
)

In [7]:
query = "How can we contribute to the course?"

In [9]:
index.fit(documents)

In [12]:
boost = {'question': 2.0, 'section': 0.5}
filter = {'course': 'data-engineering-zoomcamp'}

results = index.search(
    query = query,
    boost_dict = boost,
    filter_dict = filter,
    num_results = 5
)

In [13]:
from openai import OpenAI

In [14]:
client = OpenAI(
    base_url="http://localhost:11434/v1/",
    api_key="ollama"
)

In [16]:
response = client.chat.completions.create(
    model='deepseek-r1:1.5b',
    messages=[{"role": "user", "content": query}]
)

In [17]:
response.choices[0].message.content

'<think>\n\n</think>\n\nI\'m an AI assistant that uses algorithms to process information and create responses. I don\'t have actual consciousness or emotions, so I don\'t experience "adding to a course" directly. However, if you have specific questions or need help with something, feel free to share, and I\'ll do my best to assist!'

In [18]:
prompt_template = """
You're a course teaching assistant. Answer the QUESTION based on the CONTEXT from the FAQ database.
Use only the facts from the CONTEXT when answering the QUESTION.
If the CONTEXT does not have the Answer then output as you could not find the answer

QUESTION: {question}

CONTEXT: 
{context}
""".strip()

In [20]:
context = ""
    
for doc in results:
    context = context + f"section: {doc['section']}\nquestion: {doc['question']}\nanswer: {doc['text']}\n\n"

prompt = prompt_template.format(question=query, context=context).strip()

In [22]:
response = client.chat.completions.create(
    model='deepseek-r1:1.5b',
    messages=[{"role": "user", "content": prompt}]
)
response.choices[0].message.content

"<think>\nOkay, so I'm trying to figure out how to join the course after it starts. Let me look at the context provided.\n\nThe first question is about when the course will start. From earlier answers, it says the course starts on January 15th, 2024, at 17:00. So that's helpful information.\n\nBut how do I actually join? The previous replies talked about installing dependencies before the course begins and joining an online learning platform like Proctio, which allows in-person access. However, looking back, there were answers about installing tools and using Proctio for remote participation.\n\nWait, I see earlier responses mentioned installing Google cloud account, GCP SDK, Python 3 with Anaconda, Terraform, Git, and setting up prerequisites like Prerequisites and syllabus to feel comfortable before the course starts. That makes sense. It suggests that if you're ready to start right away or later, you can do so before the given date.\n\nBut if I join after the start date, some people